In [1]:
import sys
import os
sys.path.append('../dags')

from database import create_database_engine


# 환경 변수 설정
os.environ["POSTGRES_USER"] = "airflow"
os.environ["POSTGRES_PASSWORD"] = "airflow"
os.environ["POSTGRES_DB"] = "events"
os.environ["POSTGRES_PORT"] = "5433"

engine = create_database_engine(host="127.0.0.1")
print("Successfully connected to the database")

Successfully connected to the database


In [2]:
import pandas as pd
from datetime import datetime

# 나머지는 변수로 관리
label_table = "kind"
return_table = "abnormal_return_kind"
label_col = "label"

# 포함할 abnormal return 컬럼
abn_return_cols = [
    "abn_ret_minus_60m", "abn_ret_minus_50m", "abn_ret_minus_40m", "abn_ret_minus_30m", "abn_ret_minus_20m", "abn_ret_minus_10m",
    "abn_ret_10m", "abn_ret_20m", "abn_ret_30m", "abn_ret_40m", "abn_ret_50m", "abn_ret_60m"
]

# 포함할 cum_volume 컬럼
volume_cols = [
    "cum_volume_minus_60m", "cum_volume_minus_50m", "cum_volume_minus_40m", "cum_volume_minus_30m", "cum_volume_minus_20m", "cum_volume_minus_10m",
    "cum_volume_10m", "cum_volume_20m", "cum_volume_30m", "cum_volume_40m", "cum_volume_50m", "cum_volume_60m"
]

# SQL 컬럼 문자열을 만듬 (event_id, event_ts는 하드코딩)
sql_columns = (
    [f"ar.event_ts"] +
    [f"ar.{col}" for col in abn_return_cols] +
    [f"ar.{col}" for col in volume_cols] +
    [f"k.{label_col}"]
)
sql_columns_str = ",\n    ".join(sql_columns)

query = f"""
SELECT {sql_columns_str}
FROM {return_table} ar
JOIN "{label_table}" k ON ar.event_id = k.id
ORDER BY ar.event_ts ASC
"""

df_total = pd.read_sql(query, engine)
df_total["period_dummy"] = (
    pd.to_datetime(df_total["event_ts"], utc=True) > pd.Timestamp("2022-06-30", tz="UTC")
).astype(int)
df_total

DETAIL:  The database was created using collation version 2.36, but the operating system provides version 2.41.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE events REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.


,event_ts,abn_ret_minus_60m,abn_ret_minus_50m,abn_ret_minus_40m,abn_ret_minus_30m,abn_ret_minus_20m,abn_ret_minus_10m,abn_ret_10m,abn_ret_20m,abn_ret_30m,...,cum_volume_minus_20m,cum_volume_minus_10m,cum_volume_10m,cum_volume_20m,cum_volume_30m,cum_volume_40m,cum_volume_50m,cum_volume_60m,label,period_dummy
0,2021-01-04 10:08:00+00:00,NaN,NaN,NaN,-2.14,-2.14,-2.14,-0.45,0.17,0.05,...,37613.0,37413.0,58798.0,71353.0,81261.0,92625.0,102312.0,109293.0,0,0
1,2021-01-04 10:08:00+00:00,NaN,NaN,NaN,-2.14,-2.14,-2.14,-0.45,0.17,0.05,...,37613.0,37413.0,58798.0,71353.0,81261.0,92625.0,102312.0,109293.0,0,0
2,2021-01-04 10:08:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,-1.91,-0.66,-1.47,...,34208.0,34208.0,74063.0,77336.0,88710.0,95588.0,104470.0,106830.0,1,0
3,2021-01-04 10:14:00+00:00,NaN,NaN,NaN,NaN,NaN,-8.40,-4.90,-4.20,-3.46,...,200844.0,186392.0,863840.0,993822.0,1195841.0,1288467.0,1319037.0,1389208.0,1,0
4,2021-01-04 10:16:00+00:00,NaN,NaN,1.55,1.55,1.55,0.22,0.03,-0.83,-0.54,...,204731.0,119572.0,59246.0,121680.0,167284.0,222801.0,257635.0,316213.0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8651,2023-12-28 14:46:00+00:00,-1.89,-2.06,-1.96,-2.05,-1.92,-1.82,7.36,4.96,5.17,...,5124.0,4403.0,115783.0,141291.0,152308.0,159431.0,159431.0,159431.0,1,1
8652,2023-12-28 14:53:00+00:00,0.86,0.25,0.26,0.02,0.80,-0.08,0.24,0.24,NaN,...,17937.0,14085.0,17884.0,24052.0,34468.0,34468.0,34468.0,34468.0,1,1
8653,2023-12-28 14:56:00+00:00,1.52,1.76,1.30,0.07,0.66,0.21,0.42,-0.14,NaN,...,5502.0,2973.0,2660.0,11246.0,23002.0,23002.0,23002.0,23002.0,0,1
8654,2023-12-28 14:56:00+00:00,1.52,1.76,1.30,0.07,0.66,0.21,0.42,-0.14,NaN,...,5502.0,2973.0,2660.0,11246.0,23002.0,23002.0,23002.0,23002.0,0,1


In [3]:
df_total

,event_ts,abn_ret_minus_60m,abn_ret_minus_50m,abn_ret_minus_40m,abn_ret_minus_30m,abn_ret_minus_20m,abn_ret_minus_10m,abn_ret_10m,abn_ret_20m,abn_ret_30m,...,cum_volume_minus_20m,cum_volume_minus_10m,cum_volume_10m,cum_volume_20m,cum_volume_30m,cum_volume_40m,cum_volume_50m,cum_volume_60m,label,period_dummy
0,2021-01-04 10:08:00+00:00,NaN,NaN,NaN,-2.14,-2.14,-2.14,-0.45,0.17,0.05,...,37613.0,37413.0,58798.0,71353.0,81261.0,92625.0,102312.0,109293.0,0,0
1,2021-01-04 10:08:00+00:00,NaN,NaN,NaN,-2.14,-2.14,-2.14,-0.45,0.17,0.05,...,37613.0,37413.0,58798.0,71353.0,81261.0,92625.0,102312.0,109293.0,0,0
2,2021-01-04 10:08:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,-1.91,-0.66,-1.47,...,34208.0,34208.0,74063.0,77336.0,88710.0,95588.0,104470.0,106830.0,1,0
3,2021-01-04 10:14:00+00:00,NaN,NaN,NaN,NaN,NaN,-8.40,-4.90,-4.20,-3.46,...,200844.0,186392.0,863840.0,993822.0,1195841.0,1288467.0,1319037.0,1389208.0,1,0
4,2021-01-04 10:16:00+00:00,NaN,NaN,1.55,1.55,1.55,0.22,0.03,-0.83,-0.54,...,204731.0,119572.0,59246.0,121680.0,167284.0,222801.0,257635.0,316213.0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8651,2023-12-28 14:46:00+00:00,-1.89,-2.06,-1.96,-2.05,-1.92,-1.82,7.36,4.96,5.17,...,5124.0,4403.0,115783.0,141291.0,152308.0,159431.0,159431.0,159431.0,1,1
8652,2023-12-28 14:53:00+00:00,0.86,0.25,0.26,0.02,0.80,-0.08,0.24,0.24,NaN,...,17937.0,14085.0,17884.0,24052.0,34468.0,34468.0,34468.0,34468.0,1,1
8653,2023-12-28 14:56:00+00:00,1.52,1.76,1.30,0.07,0.66,0.21,0.42,-0.14,NaN,...,5502.0,2973.0,2660.0,11246.0,23002.0,23002.0,23002.0,23002.0,0,1
8654,2023-12-28 14:56:00+00:00,1.52,1.76,1.30,0.07,0.66,0.21,0.42,-0.14,NaN,...,5502.0,2973.0,2660.0,11246.0,23002.0,23002.0,23002.0,23002.0,0,1


In [4]:
import numpy as np

# 10, 20, ..., 60 윈도우에 대해 delta_cum_volume 계산
for m in range(10, 61, 10):
    denom = 1 + df_total[f"cum_volume_minus_{m}m"]
    num = 1 + df_total[f"cum_volume_{m}m"]
    df_total[f"delta_cum_volume_{m}m"] = np.log(num / denom)

# min-max scaling for each delta_cum_volume_{m}m to [-1, 1]
for m in range(10, 61, 10):
    col = f"delta_cum_volume_{m}m"
    x = df_total[col].values
    x_min = np.nanmin(x)
    x_max = np.nanmax(x)
    df_total[col] = 2 * (x - x_min) / (x_max - x_min) - 1

# 누적 거래량으로 시작하는 컬럼 모두 drop
cum_cols_to_drop = [col for col in df_total.columns if col.startswith("cum_volume_")]
df_total = df_total.drop(columns=cum_cols_to_drop)
df_total.filter(regex="^delta_cum_").head()


,delta_cum_volume_10m,delta_cum_volume_20m,delta_cum_volume_30m,delta_cum_volume_40m,delta_cum_volume_50m,delta_cum_volume_60m
0,0.083598,0.144072,0.145074,0.082783,0.086337,0.066539
1,0.083598,0.144072,0.145074,0.082783,0.086337,0.066539
2,0.127802,0.166488,0.168029,0.097967,0.100136,0.075245
3,0.232819,0.266594,0.269576,0.191918,0.185550,0.161737
4,-0.075673,-0.004242,0.023991,-0.013224,-0.003374,-0.005831


In [5]:
df_total.filter(regex="^delta_cum_").describe()


,delta_cum_volume_10m,delta_cum_volume_20m,delta_cum_volume_30m,delta_cum_volume_40m,delta_cum_volume_50m,delta_cum_volume_60m
count,8656.000000,8656.000000,8656.000000,8656.000000,8656.000000,8656.000000
mean,0.114017,0.144022,0.123198,0.048590,0.037697,0.006608
std,0.149567,0.142230,0.139836,0.130971,0.134232,0.133269
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,0.037704,0.070676,0.050984,-0.022058,-0.034948,-0.064903
50%,0.103214,0.137011,0.118083,0.042241,0.030176,0.001142
75%,0.186815,0.216886,0.193538,0.114304,0.102081,0.070575
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
df_total[df_total["period_dummy"] == 1].head()

,event_ts,abn_ret_minus_60m,abn_ret_minus_50m,abn_ret_minus_40m,abn_ret_minus_30m,abn_ret_minus_20m,abn_ret_minus_10m,abn_ret_10m,abn_ret_20m,abn_ret_30m,...,abn_ret_50m,abn_ret_60m,label,period_dummy,delta_cum_volume_10m,delta_cum_volume_20m,delta_cum_volume_30m,delta_cum_volume_40m,delta_cum_volume_50m,delta_cum_volume_60m
5072,2022-07-01 10:03:00+00:00,-1.18,-2.32,-2.74,-2.03,-1.61,-1.80,-1.27,-2.41,-2.36,...,-2.98,-1.98,1,1,-0.005757,0.083246,0.073672,0.028160,0.019277,-0.036871
5073,2022-07-01 10:24:00+00:00,-1.48,-2.52,-1.33,-1.91,-1.62,-1.57,0.53,2.43,1.19,...,1.38,1.72,1,1,0.188400,0.321001,0.321234,0.214157,0.150370,0.115759
5074,2022-07-01 10:26:00+00:00,-0.43,-0.29,-0.91,0.34,0.62,0.10,0.14,0.19,1.01,...,1.87,1.68,0,1,0.241785,0.170342,0.160364,0.060437,0.033450,-0.037571
5075,2022-07-01 10:43:00+00:00,0.16,-0.59,-0.81,-0.82,-0.92,-0.89,0.08,0.36,0.31,...,-0.18,0.23,0,1,0.011265,0.040839,0.022614,-0.044183,-0.056170,-0.086890
5076,2022-07-01 11:03:00+00:00,-0.91,-1.21,-1.02,-0.76,-0.13,-0.21,-0.23,0.13,0.09,...,-0.26,-0.13,1,1,0.087719,0.028894,-0.021563,-0.086223,-0.073627,-0.101968


# 회귀분석

### 📘 모형식

$$
\left| CAR_{post,i} - CAR_{pre,i} \right| = \alpha + \beta \cdot Telegram_i + \epsilon_i
$$

---

### 📊 변수 설명

**종속변수 (Dependent Variable)**  
이벤트 전후 CAR 변화폭의 절댓값:

$$
\left| CAR_{post,i} - CAR_{pre,i} \right|
$$


**설명변수 (Key Independent Variable)**  
텔레그램 도입 여부 변수:

$$
Telegram_i =
\begin{cases}
1, & \text{텔레그램 도입 이후 (After introduction)} \\
0, & \text{도입 이전 (Before introduction)}
\end{cases}
$$

---

### 📑 가설 설정

$$
\begin{aligned}
H_0 &: \beta = 0 \quad &(\text{텔레그램 도입 이후와 이전 간 CAR 변화폭의 차이가 없다.}) \\
H_1 &: \beta < 0 \quad &(\text{텔레그램 도입 이후 CAR 변화폭이 감소했다.})
\end{aligned}
$$
---

### 📈 해석

이 회귀모형은 **텔레그램 도입 여부**가 이벤트 전후 누적초과수익률(CAR) 변화폭의 크기에 미치는 영향을 분석하기 위한 것입니다.  
β < 0 이고 통계적으로 유의하다면 귀무가설이 기각되어 공시 정보가 도입 이후 시장에 보다 신속하게 반영되어 정보 비효율성이 감소했음을 시사하며, 텔레그램 공시 알림 서비스가 시장 효율성 향상에 실질적으로 기여했음을 보여줍니다.


In [7]:
# import numpy as np
# import pandas as pd
# import statsmodels.api as sm

# # df_total이 이미 존재한다고 가정
# horizons = list[int](range(1, 11))
# rows = []

# for h in horizons:
#     post_col = f"abn_ret_{h}m"
#     pre_col  = f"abn_ret_minus_{h}m"
#     if post_col not in df_total.columns or pre_col not in df_total.columns:
#         raise KeyError(f"'{post_col}' 또는 '{pre_col}' 컬럼을 찾을 수 없습니다.")

#     y = (df_total[post_col] - df_total[pre_col]).abs()
#     X = sm.add_constant(df_total["period_dummy"], has_constant="add")

#     valid = ~(y.isna() | X.isna().any(axis=1))
#     y_ = y[valid]
#     X_ = X[valid]

#     model = sm.OLS(y_, X_).fit(cov_type="HC1")

#     beta = model.params.get("period_dummy", np.nan)
#     se   = model.bse.get("period_dummy", np.nan)
#     tval = model.tvalues.get("period_dummy", np.nan)
#     pval = model.pvalues.get("period_dummy", np.nan)

#     rows.append({
#         "horizon_min": h,
#         "beta": beta,
#         "std_err": se,
#         "t_value": tval,
#         "p_value": pval,
#         "sig_0.05": "*" if (not pd.isna(pval) and pval < 0.05) else "",
#         "n_obs": int(model.nobs),
#         "r_squared": model.rsquared
#     })

# result_table = pd.DataFrame(rows).sort_values("horizon_min").reset_index(drop=True)

# # ✅ p-value 포맷 조정 (지수표기 없이 소수점 8자리, 0은 "<1e-8"로 표시)
# result_table["p_value"] = result_table["p_value"].apply(
#     lambda x: f"{x:.12f}" if (x > 0 and not pd.isna(x)) else "<1e-8"
# )

# # 보기 좋게 출력
# display_cols = ["horizon_min", "beta", "std_err", "t_value", "p_value", "sig_0.05", "n_obs", "r_squared"]
# print(result_table[display_cols].to_string(index=False))

# 방향 정합성

### 📘 모형식 정의

$$
\text{logit}^{-1}(x) = \frac{1}{1 + e^{-x}}
$$

$$
Pr(hit_{i,t} = 1) = \text{logit}^{-1}(\alpha + \beta \cdot period\_dummy_i)
$$
---

### 📊 변수 설명


이벤트 *i*, 창 *t* 에 대해 Delta CAR, post CAR 2개의 지표를 사용

$$
t \in \{10,\,20,\,\dots,\,60\}
$$

$$
\Delta CAR_{i,t} = CAR_{post,i,t} - CAR_{pre,i,t}  
$$

$$
CAR_{post,i,t}
$$

$$
label\_sign_i =
\begin{cases}
+1, & \text{긍정 (1)} \\
0, & \text{중립 (0)} \\
-1, & \text{부정 (-1)} 
\end{cases}
$$



$$
hit_{i,t} =
\begin{cases}
1, & \text{if } \operatorname{sign}(\Delta CAR_{i,t}) = label\_sign_i \\
0, & \text{otherwise}
\end{cases}
$$

---

### 📑 가설 설정

$$
\begin{aligned}
H_0 &: \beta = 0 \quad &(\text{도입 전후 차이 없음}) \\
H_1 &: \beta > 0 \quad &(\text{도입 후 정합률 상승})
\end{aligned}
$$

---

### 📈 해석

- delta CAR: β > 0 이고 **통계적으로 유의**하다면  → “텔레그램 서비스 출시 이후 과잉반응 감소 / 즉시 반영 후 빠른 수렴 → 정보의 선반영 및 사전 확산이 강화 → 정보 효율성 향상”을 의미한다고 볼 수 있다.
- Post CAR: β > 0 이고 **통계적으로 유의**하다면  → “텔레그램 서비스 출시 이후 이벤트 발표 이후의 정합률(정확도) 증가 → 시장 반응의 즉시 반영 강화 → 정보 효율성 향상”을 의미한다고 해석할 수 있다.


“중립 이벤트를 포함한 모델에서는 텔레그램 도입 이후 정합률이 유의하게 상승하였으나,
중립을 제외한 경우에는 그 효과가 약화되었다. 이는 정보 효율성 개선이
주로 가격 반응의 ‘적정성(overshooting 감소)’ 혹은 ‘비과잉 반응’ 영역에서 나타났음을 시사한다.
다시 말해, 시장이 새로운 정보를 과도하게 반응하지 않고 신속히 흡수하게 된 결과로 볼 수 있다.”

또는 요약형으로:

“효율성 개선은 방향성 예측력이 아니라, 불필요한 반응을 줄이는 형태로 나타났다.”

## 중립 포함

In [8]:
import sys
import os
sys.path.append('.')

from hypothesis_test_volume import (
    print_sample_summary_with_neutral,
    logistic_hit_delta_with_neutral_and_vol,
    logistic_hit_postCAR_with_neutral_and_vol,
    run_logistic_table
)

# === 전체 파이프라인 (중립 포함, 중립은 ΔCAR/POSTCAR 절대값 기준) ===

df = df_total.copy()
df["label_sign"] = df["label"]

df_full = print_sample_summary_with_neutral(df, "label_sign")
# 창 목록
windows = list(range(10, 61, 10))

# ΔCAR + 거래량 공변량 (중립 포함)
results_delta_w_neutral_vol = run_logistic_table(
    logistic_hit_delta_with_neutral_and_vol,
    df_full,
    windows,
    label="ΔCAR + volume (neutral incl.)",
    covar_prefix="delta_cum_volume_",  # 기본값과 동일
    add_interaction=False,             # 상호작용 보려면 True
    robust_se=True
)

# postCAR + 거래량 공변량 (중립 포함)
results_post_w_neutral_vol = run_logistic_table(
    logistic_hit_postCAR_with_neutral_and_vol,
    df_full,
    windows,
    label="postCAR + volume (neutral incl.)",
    covar_prefix="delta_cum_volume_",
    add_interaction=False,
    robust_se=True
)



=== Sample summary (including neutral) ===
Total events:        8,656
Neutrals:            2,980
  - Positive (1):    4,906
  - Negative (-1):   770


--- ΔCAR + volume (neutral incl.) 기준 결과 ---
 t beta_star    std  t_stat  p_value  odds_ratio  p_before  p_after  diff_pp  n_obs     R2  adj_R2  neutral_epsilon gamma_vol  std_vol        p_vol
10  0.1412** 0.0448   3.149 0.001639       1.152     0.591    0.625     3.36   8656 0.0018  0.0012         0.631883  0.4664**   0.1517 2.116564e-03
20   0.0972* 0.0441   2.204 0.027538       1.102     0.545    0.569     2.38   8656 0.0041  0.0036         0.710237 1.0108***   0.1544 5.900000e-11
30    0.0569 0.0439   1.296 0.194945       1.059     0.501    0.515     1.41   8656 0.0084  0.0079         0.768600 1.5580***   0.1575 0.000000e+00
40    0.0268 0.0441   0.608 0.543111       1.027     0.468    0.474     0.66   8656 0.0108  0.0103         0.837803 1.9167***   0.1690 0.000000e+00
50    0.0365 0.0446   0.818 0.413477       1.037     0.420    0.4

## 중립 제거

In [9]:
from hypothesis_test_volume import (
    print_sample_summary,
    logistic_hit_delta_with_vol,
    logistic_hit_postCAR_with_vol,
    run_logistic_table
)

# === 전체 파이프라인 ===

df = df_total.copy()
df["label_sign"] = df["label"]
df_nn = print_sample_summary(df, "label_sign")

# ΔCAR + volume (중립 제거)
results_delta_vol = run_logistic_table(
    logistic_hit_delta_with_vol,
    df_nn, windows,
    label="ΔCAR + volume (neutral removed)",
    covar_prefix="delta_cum_volume_",  # 이미 만드신 컬럼 접두사
    add_interaction=False,             # 필요 시 True
    robust_se=True
)

# postCAR + volume (중립 제거)
results_post_vol = run_logistic_table(
    logistic_hit_postCAR_with_vol,
    df_nn, windows,
    label="postCAR + volume (neutral removed)",
    covar_prefix="delta_cum_volume_",
    add_interaction=False,
    robust_se=True
)



=== Sample summary (neutral removed) ===
Total events:        8,656
Removed neutrals:    2,980
Used (non-neutral):  5,676
  - Positive (1):    4,906
  - Negative (-1):   770


--- ΔCAR + volume (neutral removed) 기준 결과 ---
 t beta_star    std  t_stat  p_value  odds_ratio  p_before  p_after  diff_pp  n_obs     R2  adj_R2 gamma_vol  std_vol  p_vol
10    0.0502 0.0580   0.866 0.386228       1.052     0.669    0.680     1.07   5676 0.0260  0.0252 2.7343***   0.2251    0.0
20    0.0225 0.0560   0.401 0.688347       1.023     0.610    0.616     0.51   5676 0.0314  0.0306 3.1575***   0.2216    0.0
30   -0.0046 0.0552  -0.084 0.933360       0.995     0.564    0.562    -0.11   5676 0.0355  0.0347 3.4303***   0.2235    0.0
40   -0.0337 0.0550  -0.613 0.539956       0.967     0.530    0.522    -0.80   5676 0.0406  0.0398 3.9617***   0.2375    0.0
50    0.0184 0.0553   0.332 0.739874       1.019     0.473    0.477     0.43   5676 0.0469  0.0461 4.1851***   0.2336    0.0
60   -0.0417 0.0557  -0.749 